<a href="https://colab.research.google.com/github/Deadspree/PointCloudDetect/blob/main/detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install open3d


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.0 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Unin

In [ ]:
!git clone https://github.com/meder411/PointNet-PyTorch

Cloning into 'PointNet-PyTorch'...
remote: Enumerating objects: 84, done.
remote: Total 84 (delta 0), reused 0 (delta 0), pack-reused 84 (from 1)
Receiving objects: 100% (84/84), 25.58 MiB | 39.09 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [ ]:
%cd PointNet-PyTorch/

/content/PointNet-PyTorch


In [ ]:
!ls

classifier_model_state.pth  dataloader.py  __init__.py	readme.md	    train_classifier.py
data			    img		   models	test_classifier.py


[Open3D WARNING] Read PLY failed: unable to open file: cleaned_bottle.ply
torch.Size([1, 2000, 3])


In [ ]:
import torch
import torch.nn as nn
import torch.autograd as grad
import torch.nn.functional as F
import numpy as np
import open3d as o3d

import sys
sys.path.append('/content/PointNet-PyTorch')
from models.pointnet_classifier import PointNetClassifier


# File paths
ply_file = "/content/cleaned_radius_4.ply"
model_path = "/content/PointNet-PyTorch/classifier_model_state.pth"

# PointNet settings
num_points = 2000
dims = 3
batch_size = 32

# **Check for CUDA availability and set device**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and preprocess the .ply file
point_cloud = o3d.io.read_point_cloud(ply_file)
points = np.asarray(point_cloud.points)
print(points.shape)
# Downsample or trim to the number of points expected by the model
if len(points) > num_points:
    points = points[:num_points, :]
elif len(points) < num_points:
# If the point cloud has fewer points, pad with zeros (or duplicate points)
    points = np.pad(points, ((0, num_points - len(points)), (0, 0)), mode='constant')

# Convert the point cloud to a torch tensor and move to device
points = torch.tensor(points, dtype=torch.float32, device=device).unsqueeze(-1)  # Add batch dimension
points = points.reshape(1,3,2000)
print(points.shape)

# Instantiate the network, move to device and load the pretrained weights
classifier = PointNetClassifier(num_points, dims).to(device).eval()  # Move model to device
classifier.load_state_dict(torch.load(model_path, map_location=device))  # Load model on device

 # Forward pass
with torch.no_grad():
    pred, _ = classifier(points)

 # Get the predicted class
_, predicted_class = torch.max(F.softmax(pred, dim=1), 1)
print(predicted_class)
predicted_class = predicted_class.item()  # Get the class index as a scalar

# Print the predicted class
print(f"Predicted class: {predicted_class}")

(16212, 3)
torch.Size([1, 3, 2000])
tensor([34], device='cuda:0')
Predicted class: 34


<ipython-input-33-b031a4a250e0>:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier.load_state_dict(torch.load(model_path, map_location=device))  # Load model on de

#34 is the index of "Bottle" label